# チューニングのポイント　その1

### 実行前処理

In [21]:
# 日本語化ライブラリ導入
!pip install japanize-matplotlib | tail -n 1

ERROR: Operation cancelled by user


In [22]:
# 共通事前処理

# 余分なワーニングを非表示にする
import warnings
warnings.filterwarnings('ignore')

# 必要ライブラリのimport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# matplotlib日本語化対応
import japanize_matplotlib

# データフレーム表示用関数
from IPython.display import display

# 表示オプション調整
# numpyの浮動小数点の表示精度
np.set_printoptions(suppress=True, precision=4)
# pandasでの浮動小数点の表示精度
pd.options.display.float_format = '{:.4f}'.format
# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)
# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14
# 乱数の種
random_seed = 123

#### サンプルデータの読み込み
乳がん疾患データ

In [23]:
# サンプルデータの読み込み
# (乳がん疾患データ)

# データのロード
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

# 入力データ: x (30次元)
# 正解データ: y
x = cancer.data
y = cancer.target

In [24]:
# サンプルデータの分割

# データ分割のパラメータ
test_size = 0.1

# データ分割
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, 
    test_size=test_size, random_state=random_seed,
    stratify=y)

# 分割後サイズ確認
print(x.shape)
print(x_train.shape)
print(x_test.shape)

(569, 30)
(512, 30)
(57, 30)


## チューニングのポイント1　アルゴリズムの選択

最初に確認したいアルゴリズムを宣言して、リストにしておきます。リストをループさせて、各アリゴリズムの学習結果を確認します。

実際は、アルゴリズム(ニューラルネットワーク)によっては、入力データを正規化する必要がありますが、決定木型モデルではこのやり方で行けます。




In [7]:
# 複数アルゴリズムで精度を比較
# 結果が同じになるようrandom_stateは同一にする

# 線形回帰
from sklearn.linear_model import LogisticRegression
algorithm1 = LogisticRegression(random_state=random_seed)

# サポートベクターマシン(カーネル)
from sklearn.svm import SVC
algorithm2 = SVC(kernel='rbf', random_state=random_seed)

# ニューラルネットワーク　
from sklearn.neural_network import MLPClassifier
algorithm3 = MLPClassifier(hidden_layer_sizes=(100,100), random_state=random_seed)

# 決定木
from sklearn.tree import DecisionTreeClassifier
algorithm4 = DecisionTreeClassifier(random_state=random_seed)

# ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier
algorithm5 = RandomForestClassifier(random_state=random_seed)

# XGBoost
from xgboost import XGBClassifier
algorithm6 = XGBClassifier(random_state=random_seed)

# アルゴリズムのリスト作成
algorithms = [algorithm1, algorithm2, algorithm3, algorithm4, 
    algorithm5, algorithm6]

In [8]:
# 複数アルゴリズムで精度比較
for algorithm in algorithms:
    
    # 訓練データで学習
    algorithm.fit(x_train, y_train)
    
    # 検証データで精度測定
    score = algorithm.score(x_test, y_test)
    
    # アルゴリズム名取得
    name = algorithm.__class__.__name__

    # 精度とアルゴリズム名表示
    print(f'score: {score:.4f}  {name}')

score: 0.9649  LogisticRegression
score: 0.8947  SVC
score: 0.9649  MLPClassifier
score: 0.9474  DecisionTreeClassifier
score: 0.9298  RandomForestClassifier
score: 0.9825  XGBClassifier


各アルゴリズムの結果を確認すると、XGBoostが一番よさそうです。
後ほど結果を検証する方法を紹介し、検証した結果と比べたいと思います。

#### 1つの例だけでは面白くないので　アイリスデータセットでもやってみましょう。


In [12]:
# ランダムフォレストのモデルを作るまで
# サンプルデータの読み込み
import seaborn as sns
df_iris = sns.load_dataset("iris")
columns_i = ['がく片長', 'がく片幅', '花弁長', '花弁幅', '種別']
df_iris.columns = columns_i
# 入力データ x
xi = df_iris[['がく片長', 'がく片幅', '花弁長', '花弁幅']]
# 正解データ y
yi = df_iris['種別']


In [13]:
# サンプルデータの分割

# データ分割のパラメータ
test_size = 0.1

# データ分割
from sklearn.model_selection import train_test_split
xi_train, xi_test, yi_train, yi_test = train_test_split(xi, yi, 
    test_size=test_size, random_state=random_seed,
    stratify=yi)

# 分割後サイズ確認
print(xi.shape)
print(xi_train.shape)
print(xi_test.shape)

(150, 4)
(135, 4)
(15, 4)


In [14]:
# 複数アルゴリズムで精度比較
for algorithm in algorithms:
    
    # 訓練データで学習
    algorithm.fit(xi_train, yi_train)
    
    # 検証データで精度測定
    score = algorithm.score(xi_test, yi_test)
    
    # アルゴリズム名取得
    name = algorithm.__class__.__name__

    # 精度とアルゴリズム名表示
    print(f'score: {score:.4f}  {name}')

score: 1.0000  LogisticRegression
score: 0.9333  SVC
score: 0.9333  MLPClassifier
score: 0.9333  DecisionTreeClassifier
score: 1.0000  RandomForestClassifier
score: 0.9333  XGBClassifier


2つぐらい例を提示しておくと記憶に残りやすい

## チューニングのポイント2　ハイパーパラメータの最適化

アルゴリズムによってハイパーパラメータは異なります。
先ほどのアルゴリズムの選択では結果の悪かった
サポートベクターカーネルのハイパーパラメータをチューニングしながら、最適化方法を確認します。

※あまり結果の悪いアルゴリズムのチューニングは行わないが、大きく改善されるためわかりやすい。

In [15]:
# デフォルトパラメータの確認
algorithm = SVC(kernel='rbf', random_state=random_seed)
print(algorithm)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=123, shrinking=True, tol=0.001,
    verbose=False)


#### チューニングするハイパーパラメータのターゲットを予め決めておく必要がある
まず、サポートベクターカーネルでチューニングするのは分類エリアの大きさを調整するgammaを調整します。
チューニング値を明示的にリストにしてループさせ、その結果をみて最適値を探っていくことになります。


In [16]:
# gammaの最適化
algorithm = SVC(kernel='rbf', random_state=random_seed)

#チューニング値を明示的にリスト化
gammas = [1, 0.1, 0.01, 0.001, 0.0001, 0.00001]

for gamma in gammas:
    algorithm.gamma = gamma
    algorithm.fit(x_train, y_train)
    score = algorithm.score(x_test, y_test)
    print(f'score: {score:.4f}  gamma: {gamma}')

score: 0.6316  gamma: 1
score: 0.6316  gamma: 0.1
score: 0.6316  gamma: 0.01
score: 0.9474  gamma: 0.001
score: 0.9474  gamma: 0.0001
score: 0.9474  gamma: 1e-05


次に、サポートベクターカーネルでチューニングするのはモデルの複雑さを調整するCを調整します。 チューニング値を明示的にリストにしてループさせ、その結果をみて最適値を探っていくことになります。

In [17]:
# Cの最適化
# gammaは先ほど調べた最適値 0.001を採用
#チューニング値を明示的にリスト化
Cs = [1,  10,  100, 1000, 10000]
for C in Cs:
    algorithm = SVC(kernel='rbf', 
        gamma=0.001, C=C,
        random_state=random_seed)
    algorithm.fit(x_train, y_train)
    score = algorithm.score(x_test, y_test)
    print(f'score: {score:.4f}  C: {C}')

score: 0.9474  C: 1
score: 0.9298  C: 10
score: 0.9298  C: 100
score: 0.9298  C: 1000
score: 0.9298  C: 10000


In [20]:
# 複数アルゴリズムで精度を比較
# 結果が同じになるようrandom_stateは同一にする
# 線形回帰
algorithm1 = LogisticRegression(random_state=random_seed)

# サポートベクターマシン(カーネル)
algorithm2 =  SVC(kernel='rbf', gamma=0.001, C=1, random_state=random_seed)

# ニューラルネットワーク　
algorithm3 = MLPClassifier(hidden_layer_sizes=(100,100), random_state=random_seed)

# 決定木
algorithm4 = DecisionTreeClassifier(random_state=random_seed)

# ランダムフォレスト
algorithm5 = RandomForestClassifier(random_state=random_seed)

# XGBoost
algorithm6 = XGBClassifier(random_state=random_seed)

# アルゴリズムのリスト作成
algorithms = [algorithm1, algorithm2, algorithm3, algorithm4, 
    algorithm5, algorithm6]

# 複数アルゴリズムで精度比較
for algorithm in algorithms:
    
    # 訓練データで学習
    algorithm.fit(x_train, y_train)
    
    # 検証データで精度測定
    score = algorithm.score(x_test, y_test)
    
    # アルゴリズム名取得
    name = algorithm.__class__.__name__

    # 精度とアルゴリズム名表示
    print(f'score: {score:.4f}  {name}')

score: 0.9649  LogisticRegression
score: 0.9474  SVC
score: 0.9649  MLPClassifier
score: 0.9474  DecisionTreeClassifier
score: 0.9298  RandomForestClassifier
score: 0.9825  XGBClassifier


### これでどのアルゴリズムも甲乙つけがたくなったと思います。この結果をもとにさらに検証を行うことで、どのアルゴリズムを採用するかを決めることになります。
次は、代表的な検証方法について、紹介し上記の結果と検証結果がどれくらい異なるかを確認します。
